# 讀取資料

In [1]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

df2 = pd.read_csv("titanic_deposit.csv")
print(df2.shape)
print(df2.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']
(887, 2)
['Name', 'Deposit']


In [2]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [3]:
df2.head()

,Name,Deposit
0,Mr. Owen Harris Braund,23.46
1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,12.58
2,Miss. Laina Heikkinen,445.31
3,Mrs. Jacques Heath (Lily May Peel) Futrelle,456.45
4,Mr. William Henry Allen,564.10


# SQL 語法查詢CSV

In [4]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False, if_exists='replace')
df2.to_sql("deposit", engine, index=False, if_exists='replace')

887

In [5]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age < 5;"))
print(db.run("SELECT * FROM deposit WHERE Deposit > 100;"))

sqlite
['deposit', 'titanic']
[(0, 3, 'Master. Gosta Leonard Palsson', 'male', 2.0, 3, 1, 21.075), (1, 3, 'Miss. Marguerite Rut Sandstrom', 'female', 4.0, 1, 1, 16.7), (0, 3, 'Master. Eugene Rice', 'male', 2.0, 4, 1, 29.125), (1, 2, 'Miss. Simonne Marie Anne Andree Laroche', 'female', 3.0, 1, 2, 41.5792), (0, 3, 'Master. Harald Skoog', 'male', 4.0, 3, 2, 27.9), (1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Miss. Ellis Anna Maria Andersson', 'female', 2.0, 4, 2, 31.275), (1, 3, 'Miss. Anna Peter', 'female', 2.0, 1, 1, 22.3583), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (0, 3, 'Master. Arthur Rice', 'male', 4.0, 4, 1, 29.125), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 3, 'Miss. Luise Gretchen Kink-Heilmann', 'female', 4.0, 0, 2, 22.025), (1, 2, 'Master. Michel M Navratil', 'male', 3.0, 1, 1, 26.0), (0, 3, 'Miss. Telma Matilda Strom', 'female', 2

# SQL Agent

In [8]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model='llama3.2:3b')

In [6]:

from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...


In [ ]:
agent_executor.invoke({"input": "what's the average deposit of survivors"})

In [ ]:
agent_executor.invoke({"input": "what are the names of the survivors whose deposit is below 100"})

# 用 llm 操作 pandas dataframe

In [6]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("titanic.csv")
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['Fare'].mean()")

32.30542018038331

In [11]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
response = llm_with_tools.invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-01-05T09:44:47.6377956Z', 'done': True, 'done_reason': 'stop', 'total_duration': 382772697900, 'load_duration': 40423300, 'prompt_eval_count': 218, 'prompt_eval_duration': 258000000, 'eval_count': 33, 'eval_duration': 382471000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-e413262b-296e-4f38-bbd5-bfc1d2eab479-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "import pandas as pd; df[['Age', 'Fare']].corr()"}, 'id': 'a60a2589-1700-4c4f-99f5-6c75c0070a8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 218, 'output_tokens': 33, 'total_tokens': 251})

In [12]:
response.tool_calls

[{'name': 'python_repl_ast',
  'args': {'query': "import pandas as pd; df[['Age', 'Fare']].corr()"},
  'id': 'a60a2589-1700-4c4f-99f5-6c75c0070a8c',
  'type': 'tool_call'}]

In [13]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
(llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)

{'query': "df['Age'].corr(df['Fare'])"}

In [15]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

\`\`\`
{df.head().to_markdown()}
\`\`\`

Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llm_with_tools | parser
code_chain.invoke({"question": "What's the correlation between age and fare"})

In [16]:
code_chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You have access to a pandas dataframe `df`. Here is the output of `df.head().to_markdown()`:\n\n\\`\\`\\`\n|    |   Survived |   Pclass | Name                                               | Sex    |   Age |   Siblings/Spouses Aboard |   Parents/Children Aboard |    Fare |\n|---:|-----------:|---------:|:---------------------------------------------------|:-------|------:|--------------------------:|--------------------------:|--------:|\n|  0 |          0 |        3 | Mr. Owen Harris Braund                             | male   |    22 |                         1 |                         0 |  7.25   |\n|  1 |          1 |        1 | Mrs. John Bradley (Florence Briggs Thayer) Cumings | female |    38 |                         1 |                         0 | 71.2833 |\n|  2 |

In [17]:
chain = prompt | llm_with_tools | parser | tool
chain.invoke({"question": "What's the correlation between age and fare"})

'0.11232863699941616\n'